In [5]:
import urllib.request
import numpy as np
import matplotlib.pyplot as plt
import time
from SGA import SGA
from PMX import PMX
from utils import *
%load_ext autoreload
%autoreload 2
%matplotlib inline

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [6]:
qap_data = [
    "nug12",
    "nug14",
    "nug15",
    "nug16a",
    "nug16b",
    "nug17",
    "nug18",
    "nug20",
    "nug21",
    "nug22",
    "nug24",
    "nug25",
    "nug27",
    "nug28",
#     "nug30",
    "tai50a",
    "tai60a",
    "tai80a"
]

In [7]:
QAP_INSTANCE_URL = lambda x: f'http://anjos.mgi.polymtl.ca/qaplib/data.d/{x}.dat'
def get_qap_data(qap):
    qap_instance_file = urllib.request.urlopen(QAP_INSTANCE_URL(qap))

    line = qap_instance_file.readline()
    n = int(line.decode()[:-1].split()[0])
    print('Problem size: %d' % n)

    A = np.empty((n, n))
    if 'tai' not in qap:
        qap_instance_file.readline()
        
    for i in range(n):
        line = qap_instance_file.readline()
        A[i, :] = list(map(int, line.decode()[:-1].split()))

    B = np.empty((n, n))
    
    if qap != 'tai50a':
        qap_instance_file.readline()
        
    for i in range(n):
        line = qap_instance_file.readline()
        B[i, :] = list(map(int, line.decode()[:-1].split()))
    return A, B, n

def qap_objective_function(p, A, B, n):
    s = 0.0
    for i in range(n):
        s += (A[i, :] * B[p[i], p]).sum()
    return s

In [ ]:
population_size = 500
number_of_offspring = population_size
crossover_probability = 0.95
mutation_probability = 0.25
number_of_iterations = 250

for qap in qap_data:
    A, B, n = get_qap_data(qap)
    kwargs = {  
            'population_size':population_size,
            'chromosome_length': n,
            'number_of_offspring':number_of_offspring,
            'crossover_probability':crossover_probability,
            'mutation_probability':mutation_probability,
            'number_of_iterations': n*3,
            'tsp_objective_function':lambda x: qap_objective_function(x, A, B, n),
            'operator':PMX,
            'mutation':reverse_sequence_mutation,
            'local_search_mut_propability':0.001,
            'local_search_mut_K':3,
        }
    
    plot_scores(title=qap, **kwargs)